In [ ]:
# Install dependencies
!pip install torch torchvision torchaudio matplotlib scikit-learn kaggle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os
import zipfile
import shutil
import time

In [ ]:
# Upload your kaggle.json file first
from google.colab import files
files.upload()

# Move kaggle.json to the right place
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Download dataset
!kaggle datasets download -d techsash/waste-classification-data

# Unzip it
with zipfile.ZipFile("waste-classification-data.zip", "r") as zip_ref:
    zip_ref.extractall("/content/waste_data")

# Check folder structure
!ls /content/waste_data

!ls /content/waste_data/dataset
!ls /content/waste_data/DATASET

Dataset URL: https://www.kaggle.com/datasets/techsash/waste-classification-data
License(s): CC-BY-SA-4.0
 97% 414M/427M [00:00<00:00, 396MB/s]
100% 427M/427M [00:00<00:00, 456MB/s]
dataset  DATASET
DATASET
TEST  TRAIN


In [ ]:
# Dataset paths
train_dir = '/content/waste_data/DATASET/TRAIN'
test_dir = '/content/waste_data/DATASET/TEST'

# Transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.6892, 0.6416, 0.5668],
                         [0.2081, 0.2178, 0.2390])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.6892, 0.6416, 0.5668],
                         [0.2081, 0.2178, 0.2390])
])

In [ ]:
# Load data
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# Dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

from collections import Counter

# Assuming your test dataset is called `test_data`
class_counts = Counter(train_data.targets)  # or test_data.samples if needed
print(class_counts)

# If you want class names instead of numeric labels:
for idx, class_name in enumerate(train_data.classes):
    print(f"{class_name}: {class_counts[idx]}")


class_names = train_data.classes
print("Classes:", class_names)

Counter({0: 12565, 1: 9999})
O: 12565
R: 9999
Classes: ['O', 'R']


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class WasteClassifierCNN(nn.Module):
    def __init__(self):
        super(WasteClassifierCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 28 * 28, 512),  # adjusted for 224x224 input
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        # print(x.shape)  # prints only when you manually test one batch
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

model = WasteClassifierCNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
train_loss = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    train_loss.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/10], Loss: 0.4541
Epoch [2/10], Loss: 0.3922
Epoch [3/10], Loss: 0.3656
Epoch [4/10], Loss: 0.3528
Epoch [5/10], Loss: 0.3375
Epoch [6/10], Loss: 0.3229
Epoch [7/10], Loss: 0.3172
Epoch [8/10], Loss: 0.3104
Epoch [9/10], Loss: 0.3046
Epoch [10/10], Loss: 0.3020


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
model = model.to(device)

y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=train_data.classes))


[[1307   94]
 [ 133  979]]
              precision    recall  f1-score   support

           O       0.91      0.93      0.92      1401
           R       0.91      0.88      0.90      1112

    accuracy                           0.91      2513
   macro avg       0.91      0.91      0.91      2513
weighted avg       0.91      0.91      0.91      2513



In [ ]:
torch.save(model.state_dict(), "waste_cnn_from_scratch.pth")

In [ ]:
# model = WasteClassifierCNN()
# model.load_state_dict(torch.load('waste_cnn_from_scratch.pth'))
# model.eval()

# from PIL import Image
# from torchvision import transforms
# import torch

# # Path to the image you want to test
# img_path = "/content/images (4).jfif"

# # Define the same transforms as used during training (except augmentation)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.6892, 0.6416, 0.5668], [0.2081, 0.2178, 0.2390])  # your dataset stats
# ])

# # Load and preprocess image
# image = Image.open(img_path).convert('RGB')
# image = transform(image).unsqueeze(0)  # Add batch dimension

# # Predict
# with torch.no_grad():
#     outputs = model(image)
#     _, predicted = torch.max(outputs, 1)

# # Assuming your class names
# classes = ['O', 'R']
# print(f"Predicted class: {classes[predicted.item()]}")


Predicted class: O
